### Gradio를 활용 파일을 업로드해서 RAGAS를 질문 답변 Q&A csv를 생성

우선 Gradio를 활용해서 파일을 로딩한다.

In [1]:
import os
import gradio as gr
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader, TextLoader, CSVLoader

# 환경 변수 로드
load_dotenv()

last_uploaded_file = None

def load_file(file):
    global last_uploaded_file
    """파일을 업로드하면 파일 로더로 문서를 읽어들이는 함수"""
    if file is None:
        return "파일을 업로드해주세요."
    try:
        if isinstance(file, str):
            file_path = file
            file_name = os.path.basename(file_path)
        else:
            return "파일을 업로드해주세요."
        
        # 파일 타입에 따른 로더 선택
        if file_name.lower().endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif file_name.lower().endswith('.txt'):
            loader = TextLoader(file_path, encoding='utf-8')
        elif file_name.lower().endswith('.csv'):
            loader = CSVLoader(file_path)
        else:
            return f"❌ 지원하지 않는 파일 형식입니다: {file_name}"
        
        # 문서 로드
        docs = loader.load()
        last_uploaded_file = docs
        # 미리보기용으로 처음 1~2개만 표시
        preview = '\n---\n'.join(doc.page_content[:500] for doc in docs[:2])
        return f"✅ 파일 '{file_name}'이 성공적으로 로드되었습니다!\n\n미리보기:\n{preview}"
    except Exception as e:
        return f"❌ 파일 처리 중 오류가 발생했습니다: {str(e)}"

def create_interface():
    with gr.Blocks(title="파일 로더 데모") as demo:
        gr.Markdown("# 📄 파일 로더 데모 (챗봇 없음)")
        gr.Markdown("문서를 업로드하면 파일 로더로 읽어들여 미리보기를 보여줍니다.")
        file_input = gr.File(
            label="📄 문서 업로드",
            file_types=[".pdf", ".txt", ".csv"],
            type="filepath"
        )
        output = gr.Textbox(
            label="파일 로드 결과",
            interactive=False,
            lines=20
        )
        file_input.change(
            fn=load_file,
            inputs=[file_input],
            outputs=[output]
        )
    return demo

demo = create_interface()
demo.launch()

/Users/poowoohj/Documents/prj/homework-llm-skt/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


### 로딩된 파일을 RAGAS를 활용 Q&A CSV를 생성한다.

In [2]:
demo.close()
docs = last_uploaded_file

from langchain_text_splitters import RecursiveCharacterTextSplitter

# 문장을 구분하여 분할 - 정규표현식 사용 (문장 구분자: 마침표, 느낌표, 물음표 다음에 공백이 오는 경우)
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base",    # TikToken 인코더 이름
    separators=['\n\n', '\n', r'(?<=[.!?])\s+'],   # 구분자
    chunk_size=300,
    chunk_overlap=20,
    is_separator_regex=True,      # 구분자가 정규식인지 여부
    keep_separator=True,          # 구분자 유지 여부
)

chunks = text_splitter.split_documents(docs)

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# OpenAI Embeddings 모델을 로드
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# Chroma 벡터 저장소 생성하기
vector_store = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,    
    collection_name="skb_settop_box_user_guide", 
    # persist_directory="./chroma_db",
    collection_metadata = {'hnsw:space': 'cosine'}, # l2, ip, cosine 중에서 선택 
)

# 결과 확인
print(f"저장된 Document 개수: {len(vector_store.get()['ids'])}")

# LLM 설정
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

# LLM과 임베딩 모델 초기화
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini", temperature=0.2))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

#(3) Q&A Data 생성
from ragas.testset.persona import Persona

# 페르소나 정의 (다양한 관점에서 질문 생성)
personas = [
    Persona(
        name="elderly_customer", 
        role_description="SKB 세트탑 박스를 사용하는 고령 고객으로, 기술에 대한 이해도가 낮은 일반 사용자로 엉뚱한 질문을 많이 합니다. 한국어만을 사용합니다.",
    ),
    Persona(
        name="technician",    
        role_description="SKB 세트탑 박스를 설치하고 관리하는 기술자로, 세트탑 박스의 기능과 사용법에 대한 이해도가 높습니다. 한국어만을 사용합니다.",
    ),
]

from ragas.testset import TestsetGenerator

# TestsetGenerator 생성
generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings, persona_list=personas)

# 합성 데이터 생성
dataset = generator.generate_with_langchain_docs(chunks, testset_size=50)
# 데이터 저장
dataset.to_pandas().to_csv('./data/qa_dataset.csv', index=False)
dataset.to_pandas()

Closing server running on port: 7860
저장된 Document 개수: 4


Generating Samples: 100%|██████████| 56/56 [00:14<00:00,  3.95it/s]


,user_input,reference_contexts,reference,synthesizer_name
0,SK브로드밴드에서 제공하는 SKB 셋톱박스의 주요 기능과 설치 방법에 대해 자세히 ...,[SKB 셋톱박스 사용자 가이드\n\n1. 셋톱박스 개요\nSKB 셋톱박스는 SK브...,"SK브로드밴드에서 제공하는 SKB 셋톱박스는 디지털 방송 수신기이며, TV에서 다양...",single_hop_specifc_query_synthesizer
1,HDMI 이거 뭐야 TV랑 어떻게 해?,[SKB 셋톱박스 사용자 가이드\n\n1. 셋톱박스 개요\nSKB 셋톱박스는 SK브...,HDMI 케이블은 TV와 셋톱박스를 연결하는 케이블입니다. 이걸로 연결해야 TV에서...,single_hop_specifc_query_synthesizer
2,"에이치디엠아이 케이블이 뭔지 잘 모르겠는데, 이 HDMI라는 게 SKB 셋톱박스랑 ...",[SKB 셋톱박스 사용자 가이드\n\n1. 셋톱박스 개요\nSKB 셋톱박스는 SK브...,HDMI 케이블은 SKB 셋톱박스와 TV를 연결하는 데 사용하는 케이블입니다. SK...,single_hop_specifc_query_synthesizer
3,SK브로드밴드에서 제공하는 SKB 셋톱박스의 주요 기능은 무엇인가요?,[SKB 셋톱박스 사용자 가이드\n\n1. 셋톱박스 개요\nSKB 셋톱박스는 SK브...,"SKB 셋톱박스는 SK브로드밴드에서 제공하는 디지털 방송 수신기로, TV에서 다양한...",single_hop_specifc_query_synthesizer
4,SKB 셋톱박스는 무슨 기능들이 있나요?,[SKB 셋톱박스 사용자 가이드\n\n1. 셋톱박스 개요\nSKB 셋톱박스는 SK브...,"SKB 셋톱박스는 디지털 방송 수신, VOD 서비스 이용, 녹화 기능, 인터넷 연결...",single_hop_specifc_query_synthesizer
5,에이치디엠아이를 뭐로 바꿔야 티비에 셋톱박스 화면이 나오는지 자세히 알려주세요. 제...,[3.2 초기 설정\n- 셋톱박스 전원을 켭니다\n- TV 입력을 HDMI로 변경합...,셋톱박스 전원을 켠 후에 TV의 입력을 HDMI로 변경해야 합니다. 그러면 화면에 ...,single_hop_specifc_query_synthesizer
6,셋톱박스 초기 설정을 완료하려면 어떤 절차를 따라야 하나요?,[3.2 초기 설정\n- 셋톱박스 전원을 켭니다\n- TV 입력을 HDMI로 변경합...,셋톱박스 초기 설정을 완료하려면 먼저 셋톱박스 전원을 켜고 TV 입력을 HDMI로 ...,single_hop_specifc_query_synthesizer
7,셋톱박스 켜고 나서 HDMI 이거 뭐야? TV에 HDMI로 바꾼다는데 그거 어떻게 ...,[3.2 초기 설정\n- 셋톱박스 전원을 켭니다\n- TV 입력을 HDMI로 변경합...,셋톱박스 전원을 켜고 나서 TV의 입력을 HDMI로 변경해야 합니다. TV 리모컨에...,single_hop_specifc_query_synthesizer
8,셋톱박스 설치할 때 HDMI 어떻게 사용해요? 초기 설정에서 HDMI 입력 바꾸는 ...,[3.2 초기 설정\n- 셋톱박스 전원을 켭니다\n- TV 입력을 HDMI로 변경합...,셋톱박스 전원을 켜고 TV 입력을 HDMI로 변경합니다. 그 다음 화면에 나타나는 ...,single_hop_specifc_query_synthesizer
9,셋톱박스를 설치할 때 TV 입력을 HDMI로 변경하는 이유와 초기 설정 방법은 무엇...,[3.2 초기 설정\n- 셋톱박스 전원을 켭니다\n- TV 입력을 HDMI로 변경합...,셋톱박스 전원을 켠 후 TV 입력을 HDMI로 변경해야 합니다. 그 다음 화면에 나...,single_hop_specifc_query_synthesizer
